In [1]:
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from imblearn.ensemble import BalancedRandomForestClassifier
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score, recall_score
import joblib
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('AR Scores.csv', sep = ',', index_col=0)
accept_df = pd.read_csv('input_file_2.csv', sep = ',', index_col=0)

C:\Users\joshy\AppData\Roaming\Python\Python38\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
data['issue_d'] = pd.to_datetime(data['issue_d'])
accept_df['issue_d'] = pd.to_datetime(accept_df['issue_d'])

data = data.loc[data['issue_d'] < '2016-1-1']
accept_df = accept_df.loc[accept_df['issue_d'] < '2016-1-1']

# accept_df = accept_df.drop(accept_df[accept_df['purpose'] == 'educational'].index)

In [4]:
rejects_bad = data.loc[data["rejected"] ==1]
rejects_good = data.loc[data["rejected"] ==1]

In [5]:
rejects_good["score"] = 1 - rejects_good["score"]
rejects_good["charged_off"] = 0

rejects_bad["charged_off"] = 1

<ipython-input-5-1830eb165ca7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rejects_good["score"] = 1 - rejects_good["score"]
<ipython-input-5-1830eb165ca7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rejects_good["charged_off"] = 0
<ipython-input-5-1830eb165ca7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [6]:
ri_df = pd.concat([rejects_bad, rejects_good])

In [7]:
numerical_cols = ['loan_amnt', 'emp_length', 'dti']

accept_df = accept_df[['loan_amnt', 'issue_d', 'emp_length', 'dti','purpose','charged_off']]
accept_df['score'] = 1

all_cols = list(accept_df.columns)
all_cols.remove("charged_off")

In [8]:
# Creating the test set to represent new applications, no synthetic data.

train_df = accept_df.loc[accept_df['issue_d'] < accept_df['issue_d'].quantile(0.90)]
test_df = accept_df.loc[accept_df['issue_d'] >= accept_df['issue_d'].quantile(0.90)]

train_df = pd.concat([train_df, ri_df])

In [9]:
train_df = pd.get_dummies(train_df, prefix=["purp"], columns=['purpose'], drop_first=False)
test_df = pd.get_dummies(test_df, prefix=["purp"], columns=['purpose'], drop_first=False)

In [10]:
train_df.drop(['issue_d',"rejected"], axis=1, inplace=True)
test_df.drop(['issue_d'], axis=1, inplace=True)

In [11]:
missing_cols = list(set(train_df) - set(test_df))
test_df[missing_cols] = 0
print(list(set(test_df) - set(train_df)))

[]


In [12]:
train_df['emp_length'].fillna(train_df['emp_length'].mean(), inplace = True)
test_df['emp_length'].fillna(test_df['emp_length'].mean(), inplace=True)
print('null values      ', train_df.isnull().sum())

null values       loan_amnt                  0
emp_length                 0
dti                        0
charged_off                0
score                      0
purp_Business              0
purp_car                   0
purp_credit_card           0
purp_debt_consolidation    0
purp_educational           0
purp_home_improvement      0
purp_house                 0
purp_major_purchase        0
purp_medical               0
purp_moving                0
purp_other                 0
purp_renewable_energy      0
purp_small_business        0
purp_vacation              0
purp_wedding               0
dtype: int64


In [13]:
scaler = StandardScaler(copy=False)

train_df[numerical_cols] = scaler.fit_transform(train_df[numerical_cols], train_df['charged_off'])
test_df[numerical_cols] = scaler.transform(test_df[numerical_cols])
print(scaler.mean_)

[1.27977712e+04 1.61140444e+00 2.26103793e+01]


In [14]:
train_df_y = train_df['charged_off']
train_df_x = train_df.drop(['charged_off',"score"], axis=1)
train_weights = train_df['score']

test_df_x = test_df.drop(['charged_off',"score"], axis=1)
test_df_y = test_df['charged_off']
test_weights = test_df["score"]

In [15]:
model = joblib.load("./random_forest.joblib")
model_2 = BalancedRandomForestClassifier(**model.get_params())
model_2.set_params(max_depth=12)
print(model_2.get_params())
model_2.fit(train_df_x, train_df_y, sample_weight=train_weights)

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 12, 'max_features': 'log2', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'replacement': False, 'sampling_strategy': 'auto', 'verbose': 0, 'warm_start': False}


BalancedRandomForestClassifier(criterion='entropy', max_depth=12,
                               max_features='log2', random_state=42)

In [16]:
# 10 fold cross roc_auc
cv_scores_train = cross_val_score(estimator= model_2, X=train_df_x, y=train_df_y, scoring='roc_auc', cv=5, n_jobs=-1)
print("(Training Set) The cross-validation AUC for optimised Random Forest model after a 10 fold cross validation: %.3f (%.3f)" % (mean(cv_scores_train), std(cv_scores_train)))

cv_scores_test = cross_val_score(estimator= model_2, X=test_df_x, y=test_df_y, scoring='roc_auc', cv=5, n_jobs=-1)
print("(Test Set) The cross-validation AUC for optimised Random Forest model after a 10 fold cross validation: %.3f (%.3f)" % (mean(cv_scores_test), std(cv_scores_test)))

(Training Set) The cross-validation AUC for optimised Random Forest model after a 10 fold cross validation: 0.950 (0.018)
(Test Set) The cross-validation AUC for optimised Random Forest model after a 10 fold cross validation: 0.616 (0.007)
